### Prepare raw data

Download html files from site

In [1]:
#!wget -r -k -l 7 -p -E -nc https://povar.ru/ratingCountratingCount
#!rm -rf ./chroma_db

### Scrap data

In [2]:
from utils import make_data_set, read_recipes_pkl, save_recipes_pkl, make_recipes,filter_recipe
from Recipe import Recipe, RecipesProject

MAIN_DIR = "/home/zaderu/Documents/MLHS/LLM Project/data/povar.ru/recipes/"
FILE_NAME = "/workspace/llm_project/raw_recipes/povar_recipes.pickle"

/workspace/Recipes/rec_proj_venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# povar_recipes = make_data_set(MAIN_DIR)
# save_recipes_pkl(povar_recipes, FILE_NAME)

povar_recipes = read_recipes_pkl(FILE_NAME)

Collected 66349 recipes


In [4]:
recipes_list = make_recipes(povar_recipes, output_file="./Recipe_recipes.pickle")
recipes_list = filter_recipe(recipes_list, max_steps = 10, max_min=120, min_rating=4.3, min_votes=5)

Total 66349 recipes is processed.
16052 recipes is selected.


In [5]:
# # Chroma DB

# from build_chroma_db import build_chroma_db, requst_chroma_db
# import torch

# device = "cuda" if torch.cuda.is_available() else "cpu"
# vector_store, tokenizer = build_chroma_db(
#     recipes_list,
#     embedding_model="ai-forever/sbert_large_nlu_ru", #"nomic-ai/nomic-embed-text-v1.5", #"IlyaGusev/saiga_yandexgpt_8b" 
#     device=device,
# )

In [6]:
### Knowledge graph

from build_knowledge_graph import make_tags_list, lemmatize_tags, build_knowledge_graph\
    , lemmatize, lemmatize_sentance, make_one_word_tags_list

tags = make_tags_list(recipes_list)

lemmatize_tags(tags)

one_word_tags = make_one_word_tags_list(tags)

In [7]:
G = build_knowledge_graph(recipes_list, tags)

In [8]:
rp = RecipesProject(resipes=recipes_list, knowledgeGraph=G, tags=tags, vectorStore=None, oneWordTags=one_word_tags)
# rp.add_resipes_list(recipes_list)
# rp.add_knowledge_graph(G)
# rp.add_tags(tags)
# rp.add_one_word_tags(one_word_tags)

In [9]:
max_len = 0
for tag in tags:
    if len(tags[tag]['lemma']) > max_len:
        max_len = len(tags[tag]['lemma'])
max_len # longest tag

4

### Queries

In [23]:
from build_knowledge_graph import enreach_query_with_relative_tags

query = "Пирог с яблоками и творожной начинкой"
query = "что приготовить детского мясного"

enreach_query_with_relative_tags(query, one_word_tags, )

#Убрать ризотто

['детский', 'мясо']

In [24]:
answer = rp.invoke(query, verbose=True)
len(answer[0])

New tags are added: детский мясо
 Tag (('мясо', {'stat': 2861, 'lemma': ('мясо',)})) is applied. Selected 2750 recipes
 Tag (('к мясу', {'stat': 74, 'lemma': ('мясо',)})) is applied. Selected 2750 recipes
 Tag (('детский', {'stat': 2074, 'lemma': ('детский',)})) is applied. Selected 94 recipes


94

In [25]:
for a in answer[0]:
    print(rp.knowledgeGraph.nodes[a]['name'])
for a in answer[1]:
    print(rp.knowledgeGraph.nodes[a]['name'])    

Сосиски в кляре "Осьминожки"
Котлеты под шубой
Гуляш из печени со сметаной
Фаршированные блины с мясом
Ежики в сливочном соусе
Блинчики с ливером
Слойки с сыром и сосисками
Тефтели в томатно-сливочном соусе
Караимские пирожки
Маленькие бутерброды для детей
Суп с фрикадельками без картошки
Сосиски в лаваше жареные
Зразы из фарша в духовке
Котлеты в тесте в духовке
Пицца для детей (простой рецепт)
Слойки с ветчиной и сыром
Тефтели с картофелем
Суп с фрикадельками в мультиварке
Гречневая каша по-купечески
Пюре из мяса для ребенка
Клубный сэндвич
"Волосатые" сосиски
Котлеты с начинкой
Фрикадельки для детей
Картофельная запеканка классическая
Запеканка из вермишели с фаршем
Подлива для тефтелей
Запеканка картофельная под горчичным соусом
Булочки со свининой
Простые бутерброды
Ежики с гречкой
Тесто для пирожков с сосиской
Шашлык из сосисок
Бутерброды на день рождения ребенка
Котлеты по-флотски
Закуска "Крабик"
Суп с фрикадельками и картошкой
Сардельки в слоеном тесте в духовке
Печёночные ола

In [19]:
results = requst_chroma_db(vector_store, tokenizer, recipes_list[1000].document.page_content, device)

for res in results:
    print(f"Найди {recipes_list[1000].document.page_content[:50]}\n {res.metadata['name']}\n{res.page_content} [{res.metadata}]\n\n\n\n")

NameError: name 'requst_chroma_db' is not defined